# Introduction to Wavelength Frame Multiplication

This notebook aims to explain the concept of wavelength frame multiplication (WFM),
why is it used, how it works, and what results can be expected from using WFM at a neutron beamline.

Much of the material presented here was inspired by / copied from the paper by
[Schmakat et al. (2020)](https://www.sciencedirect.com/science/article/abs/pii/S0168900220308640),
which we highly recommend to the reader, for more details on how a WFM chopper system is designed.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.ioff() # Turn of auto-showing of figures
import scipp as sc
from scipp import constants
import scippneutron as scn
import ess.wfm as wfm
import ess.choppers as ch

In [ ]:
# Note: this cell defines code to generate figures which are used below.
# This code is hidden in the documentation pages.
from matplotlib.patches import Rectangle
import inspect
import io


class Figure:
    def __init__(self, func):
        self._func = func
        self._fig = self._func()
        self._fig.set_size_inches(6.5, 4.75)

    def _ipython_display_(self):
        return self._fig.canvas._ipython_display_()

    def _repr_png_(self):
        buf = io.BytesIO()
        self._fig.savefig(buf, format='png', bbox_inches='tight')
        plt.close(self._fig)
        buf.seek(0)
        return buf.getvalue()


def figure1():
    t_P = sc.scalar(2.860e+03, unit='us')
    detector_position = sc.vector(value=[0., 0., 60.0], unit='m')
    z_det = sc.norm(detector_position).value

    fig, ax = plt.subplots()
    ax.add_patch(
        Rectangle((0, 0), t_P.value, -0.05 * z_det, lw=1, fc='grey', ec='k', zorder=10))
    # Indicate source pulse and add the duration.
    ax.text(0,
            -0.05 * z_det,
            "Source pulse ({} {})".format(t_P.value, t_P.unit),
            ha="left",
            va="top",
            fontsize=8)
    ax.plot([0, 1.0e4], [z_det] * 2, lw=3, color='grey')
    ax.text(0., z_det, 'Detector', ha='left', va='top')
    # Draw 2 neutron paths
    ax.plot([0.02 * t_P.value, 4.0e3], [0, z_det], lw=2, color='r')
    ax.plot([t_P.value, 4.0e3], [0, z_det], lw=2, color='b')
    ax.text(3.7e3, 0.5 * z_det, r'$\lambda_{1}$', ha='left', va='center', color='b')
    ax.text(1.5e3, 0.5 * z_det, r'$\lambda_{2}$', ha='left', va='center', color='r')
    ax.set_xlabel("Time [microseconds]")
    ax.set_ylabel("Distance [m]")
    ax.set_title('Figure 1')
    return fig


def figure2():
    t_P = sc.scalar(2.860e+03, unit='us')
    detector_position = sc.vector(value=[0., 0., 60.0], unit='m')
    t_0 = sc.scalar(5.0e+02, unit='us')
    t_A = (t_0 + t_P).value
    z_det = sc.norm(detector_position).value
    fig, ax = plt.subplots()
    ax.add_patch(
        Rectangle((0, 0), (t_P + t_0).value,
                  -0.05 * z_det,
                  lw=1,
                  fc='grey',
                  ec='k',
                  zorder=10))

    # Indicate source pulse and add the duration.
    ax.text(0, -0.05 * z_det, "Source pulse", ha="left", va="top", fontsize=8)
    ax.plot([0, 3.1e4], [z_det] * 2, lw=3, color='grey')
    ax.text(0., z_det, 'Detector', ha='left', va='top')

    dt = 1000.0
    z_wfm = 15.0
    xmin = 0.0
    for i in range(3):
        xmax = 3000.0 + (i * 2.0 * dt)
        ax.plot([xmin, xmax], [z_wfm] * 2, color='k')
        xmin = xmax + dt
    ax.plot([xmin, 3.1e4], [z_wfm] * 2, color='k')
    ax.text(25000.0, z_wfm, "WFM", ha='left', va='top')
    ax.plot([t_A, 5000.0], [0, z_det], color='r')
    ax.plot([2600.0, 5000.0], [0, z_det], color='b')
    ax.plot([t_A, 13000.0], [0, z_det], color='r')
    ax.plot([2400.0, 13000.0], [0, z_det], color='b')
    ax.plot([t_A, 21500.0], [0, z_det], color='r')
    ax.plot([2200.0, 21500.0], [0, z_det], color='b')

    ax.set_xlabel("Time [microseconds]")
    ax.set_ylabel("Distance [m]")
    ax.set_title('Figure 2')
    return fig


def figure3():
    x = np.linspace(0, 5.0, 100)
    a = 4.0
    b = 0.0
    c = 1.5e10
    d = 3.0
    e = 3.0
    y = 2.0 * c / (np.exp(-a * (x - b)) + 1.0) - c
    n = 60
    y2 = c * np.exp(-e * (x - d))
    y[n:] = y2[n:]
    fig, ax = plt.subplots()
    ax.plot(x, y, lw=2, color='k')

    i1 = 5
    i2 = 65
    ax.fill([x[i1]] + x[i1:i2].tolist() + [x[i2 - 1]], [0] + y[i1:i2].tolist() + [0],
            alpha=0.3)

    fs = 15

    ax.axvline(x[i1], color='k')
    ax.axvline(x[i2 - 1], color='k')
    ax.text(x[i1], 1.58e10, r' $t_{0}$', ha='left', va='top', fontsize=fs)
    ax.text(x[i2 - 1], 1.58e10, r' $t_{\rm A}$', ha='left', va='top', fontsize=fs)
    ax.plot([4.5] * 2, [0, 0.3e10], color='k')
    ax.text(4.5, 0.3e10, r'$t_{\rm B}$', ha='center', va='bottom', fontsize=fs)
    ax.annotate(text='',
                xy=(x[i1], 0.7e10),
                xytext=(x[i2 - 1], 0.7e10),
                arrowprops=dict(arrowstyle='<->'))
    ax.text(0.5 * (x[i1] + x[i2 - 1]),
            0.7e10,
            'utilised\n pulse length',
            ha='center',
            va='bottom',
            fontsize=fs)
    ax.text(0.5 * (x[i1] + x[i2 - 1]),
            0.7e10,
            r'$t_{\rm P}$',
            ha='center',
            va='top',
            fontsize=fs)

    ax.set_ylim(0., 1.6e10)
    ax.set_xlabel("Time [ms]")
    ax.set_ylabel(r"Flux density $[{\rm n/s/cm}^2]$")
    ax.set_title('Figure 3')
    return fig


def figure4():
    coords = wfm.make_fake_beamline(nframes=1,
                                    chopper_positions={
                                        "WFMC1": sc.vector(value=[0.0, 0.0, 4.5],
                                                           unit='m'),
                                        "WFMC2": sc.vector(value=[0.0, 0.0, 5.5],
                                                           unit='m')
                                    })
    coords['position'] = sc.vector(value=[0., 0., 15.], unit='m')
    ds = sc.Dataset(coords=coords)

    z_det = sc.norm(ds.coords["position"]).value
    t_0 = ds.coords["source_pulse_t_0"].value
    t_A = (ds.coords["source_pulse_length"] + ds.coords["source_pulse_t_0"]).value
    t_B = (ds.coords["source_pulse_length"] + 2.0 * ds.coords["source_pulse_t_0"]).value
    z_foc = 12.0
    tmax_glob = 1.4e4
    height = 0.02

    chopper_wfm1 = coords["chopper_wfm_1"].value
    chopper_wfm2 = coords["chopper_wfm_2"].value

    fig, ax = plt.subplots()
    ax.add_patch(
        Rectangle((0, 0), t_B, -height * z_det, lw=1, fc='lightgrey', ec='k',
                  zorder=10))
    ax.add_patch(
        Rectangle((ds.coords["source_pulse_t_0"].value, 0),
                  ds.coords["source_pulse_length"].value,
                  -height * z_det,
                  lw=1,
                  fc='grey',
                  ec='k',
                  zorder=10))

    # Indicate source pulse and add the duration.
    ax.text(ds.coords["source_pulse_t_0"].value,
            -height * z_det,
            r"$t_{0}$",
            ha="center",
            va="top",
            fontsize=8)
    ax.text(t_A, -height * z_det, r"$t_{A}$", ha="center", va="top", fontsize=8)
    ax.text(t_B, -height * z_det, r"$t_{B}$", ha="center", va="top", fontsize=8)

    z_wfm = sc.norm(0.5 * (chopper_wfm1["position"].data + chopper_wfm2["position"].data)).value
    xmin = ch.time_open(chopper_wfm1).values[0]
    xmax = ch.time_closed(chopper_wfm1).values[0]
    dt = xmax - xmin

    ax.plot([0, xmin], [z_wfm] * 2, color='k')
    ax.plot([xmax, tmax_glob], [z_wfm] * 2, color='k')
    ax.text(tmax_glob, z_wfm, "WFMC", ha='right', va='top')

    slope_lambda_max = z_wfm / (xmin - t_0)
    slope_lambda_min = z_wfm / (xmax - t_A)
    int_lambda_max = z_wfm - slope_lambda_max * xmin
    int_lambda_min = z_wfm - slope_lambda_min * xmax
    x_lambda_max = (z_det - int_lambda_max) / slope_lambda_max
    x_lambda_min = (z_det - int_lambda_min) / slope_lambda_min

    ax.plot([t_0, x_lambda_max, x_lambda_max + dt, t_0 + dt], [0.0, z_det, z_det, 0],
            color='C0')
    ax.plot([t_A, x_lambda_min, x_lambda_min - dt, t_A - dt], [0.0, z_det, z_det, 0],
            color='C2')

    x_lambda_max_foc = (z_foc - int_lambda_max) / slope_lambda_max + dt
    x_lambda_min_foc = (z_foc - int_lambda_min) / slope_lambda_min - dt
    ax.plot([0, x_lambda_min_foc], [z_foc] * 2, color='k')
    ax.plot([x_lambda_max_foc, tmax_glob], [z_foc] * 2, color='k')
    ax.text(0.0, z_foc, "FOC", ha='left', va='top')

    slope_lambda_min_prime = z_wfm / (xmin - t_B)
    slope_lambda_max_prime = z_wfm / xmax
    int_lambda_max_prime = z_wfm - slope_lambda_max_prime * xmax
    int_lambda_min_prime = z_wfm - slope_lambda_min_prime * xmin
    x_lambda_max_prime = (z_foc - int_lambda_max_prime) / slope_lambda_max_prime
    x_lambda_min_prime = (z_foc - int_lambda_min_prime) / slope_lambda_min_prime

    ax.plot([t_B, x_lambda_min_prime], [0.0, z_foc], color='k', ls='dashed', lw=1)
    ax.plot([0, x_lambda_max_prime], [0.0, z_foc], color='k', ls='dashed', lw=1)

    ax.text(x_lambda_min - dt,
            z_det,
            r'$\lambda_{\rm min}$',
            ha='right',
            va='top',
            color='C2')
    ax.text(x_lambda_max + dt,
            z_det,
            r'$\lambda_{\rm max}$',
            ha='left',
            va='top',
            color='C0')
    ax.text(x_lambda_min_prime,
            z_foc,
            r"$\lambda_{\rm min}^{'}$",
            ha='left',
            va='top',
            color='k')
    ax.text(x_lambda_max_prime,
            z_foc,
            r"$\lambda_{\rm max}^{'}$",
            ha='left',
            va='top',
            color='k')

    ax.plot([xmin] * 2, [z_wfm, z_det + 1.0], lw=1, color='k')
    ax.plot([x_lambda_min - dt] * 2, [z_det, z_det + 1.0], lw=1, color='k')
    ax.plot([x_lambda_min] * 2, [z_det, z_det + 1.0], lw=1, color='k')
    ax.plot([x_lambda_max + dt] * 2, [z_det, z_det + 1.0], lw=1, color='k')
    ax.plot([x_lambda_max] * 2, [z_det, z_det + 1.0], lw=1, color='k')

    ax.fill([t_0 + dt, t_A - dt, 3013.10], [0, 0, 3.2963],
            color='mediumpurple',
            alpha=0.3,
            zorder=-2)
    ax.fill([x_lambda_min, x_lambda_max, 4515.077], [z_det, z_det, 6.704],
            color='mediumpurple',
            alpha=0.3,
            zorder=-2)

    ax.annotate(text='',
                xy=(xmin, z_det + 0.7),
                xytext=(x_lambda_min - dt, z_det + 0.7),
                arrowprops=dict(arrowstyle='<->'))
    ax.text(0.5 * (xmin + x_lambda_min - dt),
            z_det + 0.7,
            r'$t(\lambda_{\rm min})$',
            va='bottom',
            ha='center')
    ax.text(x_lambda_min - 0.5 * dt,
            z_det + 0.7,
            r'$\Delta t$',
            va='bottom',
            ha='center')
    ax.text(x_lambda_max + 0.5 * dt,
            z_det + 0.7,
            r'$\Delta t$',
            va='bottom',
            ha='center')

    ax.plot([0, tmax_glob], [z_det] * 2, lw=3, color='grey')
    ax.text(0., z_det, 'Detector', ha='left', va='top')

    ax.grid(True, color='lightgray', linestyle="dotted")
    ax.set_axisbelow(True)
    ax.set_xlabel("Time [microseconds]")
    ax.set_ylabel("Distance [m]")
    ax.set_title('Figure 4')
    return fig


def figure5():
    coords = wfm.make_fake_beamline(nframes=1,
                                    chopper_positions={
                                        "WFMC1": sc.vector(value=[0.0, 0.0, 4.5],
                                                           unit='m'),
                                        "WFMC2": sc.vector(value=[0.0, 0.0, 5.5],
                                                           unit='m')
                                    })
    coords['position'] = sc.vector(value=[0., 0., 15.], unit='m')
    ds = sc.Dataset(coords=coords)
    z_det = sc.norm(ds.coords["position"]).value
    frames = wfm.get_frames(ds)
    fig = wfm.plot.time_distance_diagram(ds)
    ax = fig.get_axes()[0]

    chopper_wfm1 = coords["chopper_wfm_1"].value
    chopper_wfm2 = coords["chopper_wfm_2"].value
    z_wfm = sc.norm(0.5 * (chopper_wfm1["position"].data + chopper_wfm2["position"].data)).value
    xmax = ch.time_closed(chopper_wfm1).values[0]
    z_foc = 12.0

    ax.plot([xmax] * 2, [z_wfm, z_det + 1.0], lw=1, color='k')
    ax.plot([0, frames["time_max"].values[0]], [z_wfm] * 2,
            lw=1,
            color='k',
            ls='dotted')
    ax.text(frames["time_max"].values[0],
            z_wfm,
            r'$z_{\rm WFM}$',
            ha='left',
            va='center')

    ax.plot([0, 5770.5], [z_foc] * 2, color='k')
    ax.plot([9578.9, frames["time_max"].values[0]], [z_foc] * 2, color='k')
    ax.text(frames["time_max"].values[0], z_foc, 'FOC', ha='right', va='bottom')

    ax.plot([(frames["time_min"] + frames["delta_time_min"]).values[0]] * 2,
            [z_det, z_det + 1.0],
            lw=1,
            color='k')
    ax.plot([frames["time_min"].values[0]] * 2, [z_det, z_det + 1.0], lw=1, color='k')
    ax.plot([(frames["time_max"] - frames["delta_time_max"]).values[0]] * 2,
            [z_det, z_det + 1.0],
            lw=1,
            color='k')
    ax.plot([frames["time_max"].values[0]] * 2, [z_det, z_det + 1.0], lw=1, color='k')

    xmid = (0.5 * ((frames["time_min"] + frames["time_min"] +
                    frames["delta_time_min"]).data)).values[0]
    ax.plot([xmid] * 2, [z_det, z_det + 0.5], lw=1, color='k')

    ax.annotate(text='',
                xy=(xmax, z_det + 0.4),
                xytext=(xmid, z_det + 0.4),
                arrowprops=dict(arrowstyle='<->'))
    ax.text(0.5 * (xmax + frames["time_min"].values),
            z_det + 0.4,
            r'$t(\lambda_{N})$',
            va='bottom',
            ha='center')
    ax.text(xmid,
            z_det + 1.0,
            r'$\Delta t(\lambda_{N})$',
            va='bottom',
            ha='center',
            color='C2')
    ax.text((0.5 * ((frames["time_max"] + frames["time_max"] -
                     frames["delta_time_max"]).data)).values,
            z_det + 1.0,
            r'$\Delta t(\lambda_{N+1})$',
            va='bottom',
            ha='center',
            color='C0')
    ax.plot([xmax, xmid], [z_wfm, z_det], lw=1, ls='dashed', color='k')
    ax.text(frames['time_min'].values,
            z_det,
            r'$\lambda_{N}$   ',
            ha='right',
            va='top',
            color='C2')
    ax.text(frames['time_max'].values,
            z_det,
            r'$\lambda_{N+1}$',
            ha='left',
            va='top',
            color='C0')
    ax.annotate(text='',
                xy=(0, 14),
                xytext=(xmax, 14),
                arrowprops=dict(arrowstyle='<->'))
    ax.text(0.5 * xmax, 14, r'$t_{\rm WFM}(N)$', va='top', ha='center')

    ax.lines[4].set_color('C2')
    ax.patches[2].set_color('mediumpurple')
    ax.set_xlim(-400, 12500)
    ax.set_title('Figure 5')
    return fig


def figure6():
    coords = wfm.make_fake_beamline(nframes=2,
                                    chopper_positions={
                                        "WFMC1": sc.vector(value=[0.0, 0.0, 4.5],
                                                           unit='m'),
                                        "WFMC2": sc.vector(value=[0.0, 0.0, 5.5],
                                                           unit='m')
                                    })
    coords['position'] = sc.vector(value=[0., 0., 15.], unit='m')
    ds = sc.Dataset(coords=coords)
    frames = wfm.get_frames(ds)
    fig = wfm.plot.time_distance_diagram(ds)
    ax = fig.get_axes()[0]

    chopper_wfm1 = coords["chopper_wfm_1"].value
    chopper_wfm2 = coords["chopper_wfm_2"].value
    z_wfm = sc.norm(0.5 * (chopper_wfm1["position"].data + chopper_wfm2["position"].data)).value
    z_det = sc.norm(ds.coords["position"]).value

    ax.plot([0, frames["time_max"].values[-1]], [z_wfm] * 2,
            lw=1,
            color='k',
            ls='dotted')
    ax.text(frames["time_max"].values[-1],
            z_wfm,
            r'$z_{\rm WFM}$',
            ha='left',
            va='center')

    ax.plot([(frames["time_min"] + frames["delta_time_min"]).values] * 2,
            [z_det, z_det + 1.0],
            lw=1,
            color='k')
    ax.plot([frames["time_min"].values] * 2, [z_det, z_det + 1.0], lw=1, color='k')
    ax.plot([(frames["time_max"] - frames["delta_time_max"]).values] * 2,
            [z_det, z_det + 1.0],
            lw=1,
            color='k')
    ax.plot([frames["time_max"].values] * 2, [z_det, z_det + 1.0], lw=1, color='k')

    xmid_min = (0.5 * ((frames["time_min"] + frames["time_min"] +
                        frames["delta_time_min"]).data)).values
    xmid_max = (0.5 * ((frames["time_max"] + frames["time_max"] -
                        frames["delta_time_max"]).data)).values

    ax.text(xmid_min[0],
            z_det + 1.0,
            r'$\lambda_{N=1}$',
            va='bottom',
            ha='center',
            color='C2')
    ax.text(xmid_max[0],
            z_det + 1.0,
            r'$\lambda_{2}$',
            va='bottom',
            ha='center',
            color='C0')
    ax.text(xmid_min[1],
            z_det + 1.0,
            r'$\lambda_{2}$',
            va='bottom',
            ha='center',
            color='C0')
    ax.text(xmid_max[1],
            z_det + 1.0,
            r'$\lambda_{3}$',
            va='bottom',
            ha='center',
            color='C1')

    ax.lines[6].set_color('C2')
    ax.lines[8].set_color('C0')
    ax.patches[2].set_color('mediumpurple')
    ax.patches[5].set_color('grey')
    ax.set_title('Figure 6')
    return fig


Figure1 = Figure(figure1)
Figure2 = Figure(figure2)
Figure3 = Figure(figure3)
Figure4 = Figure(figure4)
Figure5 = Figure(figure5)
Figure6 = Figure(figure6)

if sc.plotting.is_doc_build:
    setattr(Figure1, "_ipython_display_", None)
    setattr(Figure2, "_ipython_display_", None)
    setattr(Figure3, "_ipython_display_", None)
    setattr(Figure4, "_ipython_display_", None)
    setattr(Figure5, "_ipython_display_", None)
    setattr(Figure6, "_ipython_display_", None)

<div class="alert alert-info">

**Note**

The source code used to create the figures below can be viewed by downloading this notebook
(this is hidden from the documentation pages for the sake of clarity).

</div>

## The long ESS pulse

Instruments at a pulsed neutron source,
assuming an idealized rectangular source pulse in time of length $t_{P}$,
have a resolution given by
$$\frac{\Delta \lambda}{\lambda} = \frac{t_{P}}{t} = \frac{t_{P}}{\alpha \lambda z_{\rm det}} ~, ~~~~~(1)$$
where $\lambda$ is the neutron wavelength, $t$ is time,
$\alpha = m_{\rm n}/h = 2.5278 \times 10^{-4}~{\rm s}\unicode{x212B}^{-1}{\rm m}^{-1}$
is the ratio of the neutron mass and the Planck constant,
and $z_{\rm det}$ is the distance from the source to the detector.

A natural consequence of this is that the wavelength resolution $\Delta \lambda / \lambda$
becomes finer with increasing wavelength.

This also means that the resolution is poor for a long-pulsed source such as the ESS,
compared to that of a short-pulse facility, such as ISIS.
A good way to visualize this is using a time-distance diagram,
which can represent the paths taken by the neutrons from the source to the detector.

In [ ]:
Figure1

As illustrated in Fig. 1, with a long pulse, two neutrons with very different wavelengths
($\lambda_{1} < \lambda_{2}$) can reach the detector at the exact same time,
if they originated from a different part of the pulse.

The problem cannot be resolved at the detector.
According to the difference between time recorded at the detector and the measured start of the source pulse,
both neutrons have the same wavelength.
The detector recording system has no way of knowing that this is not the reality or adjusting for this.
We instead look at the pulse generation to find ways to better measure the wavelength of our neutrons.

## So what is WFM anyway?

Within the concept of wavelength frame multiplication (WFM),
each source pulse is chopped into a number of sub pulses referred to as wavelength frames,
where each wavelength frame $N$ contains a subsequent part of the spectrum of the source pulse.

The main reason for using the WFM concept is to redefine the burst time $t_{P} = \Delta t$ as implied by Eq. (1), in order to match the required wavelength resolution of the experiment.
A secondary objective, or constraint of the first, is to utilise as much of the source pulse as possible.

In [ ]:
Figure2

## A closer look

### The ESS pulse shape

At a real beamline, the pulse shape is not rectangular,
but has rising and falling edges, as shown in Fig. 3.

In [ ]:
Figure3

Here we define several important quantities:

- The pulse $t_0$ is defined as the point in time at which the pulse is bright enough for the purposes of the experiment.
- $t_{\rm A}$ is the time when the flux has fallen down to a level below the required brightness.
- $t_{\rm P} = t_{\rm A} - t_{0}$ is the portion of the pulse that is used for the measurements, the analog of the pulse length for the ideal rectangular pulse above.
- $t_{\rm B}$ marks the end of the pulse; i.e. the time when the flux is considered to be effectively zero.

### Using a single WFM chopper

The effective burst time $\Delta t$ is defined by a WFM chopper (WFMC),
as illustrated in Fig. 3,
for the two limiting wavelengths $\lambda_{\rm min}$ and $\lambda_{\rm max}$ of a single wavelength frame $N$.

The wavelength frame is re-limited in a predefined time window by at least one frame overlap chopper (FOC)
that inhibits the overlap of neutrons from various frames, as indicated by the dashed lines in Fig. 3.
Their wavelength is labeled with $\lambda_{\rm min}^{'}$ and $\lambda_{\rm max}^{'}$.
The FOC is also removes undesired neutrons with the wrong wavelength that arise from the rising and falling edges
at the beginning and at the end of the source pulse.
Although their intensity is small, neutrons with an undesired wavelength would lead to an increased background.

In [ ]:
Figure4

The relative spectral resolution $\Delta \lambda / \lambda$ at the detector position $z_{\rm det}$
is defined by the burst time $\Delta t$ and the time-of-flight $t(\lambda)$ of the neutrons (see eq. (1)).
Because $\Delta t$ is independent of the wavelength for the case of a single WFM chopper disc,
$\Delta \lambda / \lambda$ depends on $\lambda$,
on the distance $z_{\rm WFM}$ of the WFM chopper from the source,
and on the detector position $z_{\rm det}$ as depicted in Fig. 4.

The WFM chopper acts as a virtual source,
reducing the effective burst time $\Delta t$,
while at the same time also reducing the effective time-of-flight to the detector from
$t(\lambda) = \alpha \lambda z_{\rm det}$ to $t(\lambda) = \alpha \lambda (z_{\rm det} - z_{\rm WFM})$.
Note that this assumes a straight line from WFM choppers to the detector.
In the case of scattering from a sample, the first branch of the flight path ($L_{1}$) would be modified,
while the secondary path ($L_{2}$) will remain unchanged.

A resolution that depends on $\lambda$ is not suited for some applications (such as imaging, reflectometry, ...)
where a constant $\Delta \lambda / \lambda$ is much more desirable.

### Using a pair of optically blind choppers

A constant wavelength resolution $\Delta \lambda / \lambda$ can be enforced by using a pair
of optically blind WFM chopper discs,
positioned at the positions $z_{\rm WFM1}$ and $z_{\rm WFM2} = z_{\rm WFM1} + \Delta z_{\rm WFM}$,
as shown in Fig. 5.

In this context, 'optically blind' indicates that the choppers have the same opening angles,
but the phase of the second chopper is shifted such that the second chopper opens exactly at
the time when the first chopper closes.
Such a setup introduces a wavelength dependence in the effective burst time
$\Delta t(\lambda) = \alpha \lambda \Delta z_{\rm WFM}$.

The time-of-flight to the detector remains the same as for the single WFM disc setup
described above ($t(\lambda) = \alpha \lambda (z_{\rm det} - z_{\rm WFM})$)
with $z_{\rm WFM} = \frac{1}{2} (z_{\rm WFM1} + z_{\rm WFM2})$
now representing the center position of the WFM chopper pair from the source.

The resolution for an idealized instrument with infinitesimally small beam cross-section can again be
calculated using Eq. (1):

$$\frac{\Delta \lambda}{\lambda} = \frac{\Delta t(\lambda)}{t(\lambda)} = \frac{\Delta z_{\rm WFM}}{z_{\rm det} - z_{\rm WFM}} ~. ~~~~~(2)$$

Because the term on the right hand side of Eq. (2) is constant,
the resolution becomes independent of the wavelength for an optically blind WFM chopper system.

In [ ]:
Figure5

In a real WFM chopper system,
more than one frame can be realized by equipping the chopper with additional windows such that the full
band width that fits in the time period between two subsequent source pulses at the detector position is used.
This is illustrated in Fig. 6.

The frame overlap chopper defines the bandwidth of the frame $N$ in order to suppress cross-talk
from neighboring wavelength frames,
as illustrated in Fig. 6 for two subsequent wavelength frames $N = 1$ and $N = 2$.
They are designed such that $\lambda_{N, {\rm max}} = \lambda_{N+1, {\rm min}}$ yield a continuous spectrum
at the detector.
The concept of wavelength frame multiplication implies that no data can be recorded in the
time window between two subsequent wavelength frames.

In [ ]:
Figure6

An additional feature of the WFM concept is the possibility to adjust the wavelength resolution by simply
changing the distance $\Delta z_{\rm WFM}$ between the WFM chopper discs (e.g. by using a motorized linear stage).
According to Eq. (2),
reducing $\Delta z_{\rm WFM}$ leads to a finer wavelength resolution at the cost of
intensity by effectively reducing the time window in which the neutrons can pass.

## A short example

We now proceed to illustrate data processing at a WFM beamline in the form of a short example.

### Create a beamline

We first create a beamline with two WFM choppers and 6 wavelength frames,
using the `wfm.make_fake_beamline` helper utility.
The detector is placed 60 m from the source.

In [ ]:
coords = wfm.make_fake_beamline(nframes=6)
ds = sc.Dataset(coords=coords)
wfm.plot.time_distance_diagram(ds)

The properties of the frames (boundaries in time and wavelength) are computed using the `wfm.get_frames` function

In [ ]:
frames = wfm.get_frames(ds)
frames

### Create some neutrons

We create 6 neutrons with known wavelengths, one for each frame.
We choose the values for the wavelengths based on the limits given in the `frames` information above.

In [ ]:
wavelengths = sc.array(dims=['wavelength'],
                       values=[1.5, 3.0, 4.5, 6.0, 7.0, 8.25],
                       unit='angstrom')

We assume that all 6 neutrons originated half-way through the source pulse,
and we can thus calculate the times at which they hit the detector.

In [ ]:
# Neutron mass to Planck constant ratio
alpha = sc.to_unit(constants.m_n / constants.h, 's/m/angstrom')
# Distance between the detector pixel and the source
dz = sc.norm(coords['position'] - coords['source_position'])
# Compute arrival times, in microseconds
arrival_times = sc.to_unit(
    alpha * dz * wavelengths,
    'us') + coords['source_pulse_t_0'] + (0.5 * coords['source_pulse_length'])
arrival_times

### Wrap the neutron counts and the beamline into a DataArray

We make a data array that contains the beamline information and a histogram of the neutrons over the time dimension.

In [ ]:
tmin = sc.min(arrival_times)
tmax = sc.max(arrival_times)
dt = 0.1 * (tmax - tmin)
coords['time'] = sc.linspace(dim='time',
                             start=(tmin - dt).value,
                             stop=(tmax + dt).value,
                             num=2001,
                             unit=dt.unit)
counts, _ = np.histogram(arrival_times.values, bins=coords['time'].values)
da = sc.DataArray(coords=coords,
                  data=sc.array(dims=['time'], values=counts, unit='counts'))
da

In [ ]:
da.plot()

### Stitch the frames

The `time` coordinate in our data represents the time between the source $t_{0}$
and the time when the neutron hits the detector.
However, the WFM choppers are now acting as the new source choppers.
This means that the neutron time-of-flight, which will be used to compute the neutron wavelengths,
is now defined as the time between when the neutron crossed the WFM choppers and when it hit the detector
(see Fig. 5).

Because we only know the time at which the neutron arrived at the detector, not when it left the source,
the most sensible value to use as time the neutron passed through the WFM choppers is the mid-point (in time)
between the WFM chopper openings, in each frame.
This is represented by $t_{\rm WFM}(N)$ in Fig. 5.

By using the start and end detector arrival time for each frame contained in the `frames` Dataset,
we extract all the neutrons in a given frame $N$
and subtract $t_{\rm WFM}(N)$ (also found in `frames`) from the time coordinate in that frame.

Finally, we then merge (or rebin) the neutrons from all the frames onto a common time-of-flight axis.

All this is performed in a single operation using the `wfm.stitch` function:

In [ ]:
stitched = wfm.stitch(frames=frames, data=da, dim='time', bins=2001)
stitched.plot()

The resulting coordinate is now time-of-flight (`tof`),
and we can use `scippneutron` to convert the time-of-flight to wavelength.

In [ ]:
from scippneutron.tof.conversions import beamline, elastic
graph = {**beamline(scatter=False), **elastic("tof")}
wav = stitched.transform_coords("wavelength", graph=graph)
wav

In [ ]:
wav.plot()

Zooming in on the first spike, we notice that something is not quite right: the original wavelength was 1.5 &#8491; but on the figure below is closer to 1.46 &#8491;.

In [ ]:
wav["wavelength", (1.2*sc.units.angstrom):(1.7*sc.units.angstrom)].plot()

Now the WFM method guarantees a constant $\Delta\lambda / \lambda$,
and so it is not surprising to see the final reduced wavelength not exactly matching the original.
The question is: is the error within $\Delta\lambda / \lambda$?

In [ ]:
# Distance between WFM choppers
dz_wfm = sc.norm(ds.coords["chopper_wfm_2"].value["position"].data - ds.coords["chopper_wfm_1"].value["position"].data)
# Delta_lambda  / lambda
dlambda_over_lambda = dz_wfm / sc.norm(coords['position'] -
                                       frames['wfm_chopper_mid_point'].data)
(1.5*sc.units.angstrom) * dlambda_over_lambda

At 1.5 &#8491;, the resolution is 0.0127 &#8491;, which is smaller than the offset we observe above.
In fact, we can perform a quick check using Scipp's label-based slicing, to verify that the sum of the counts
in a region $\Delta\lambda$ wide around the original wavelength should be equal to 1:

In [ ]:
for i in range(len(wavelengths)):
    lam = wavelengths["wavelength", i]
    half_dlam = 0.5 * dlambda_over_lambda * lam
    print("Lambda:", lam, ", count in range:",
          sc.sum(wav['wavelength', lam - half_dlam:lam + half_dlam]).value)

This reveals that the reduced wavelengths for the first two neutrons do not agree within the required precision.
We go back to our time-distance diagram and look at the paths taken by our 6 neutrons (plotted in red).

In [ ]:
fig6 = wfm.plot.time_distance_diagram(da)
ax6 = fig6.get_axes()[0]

for i in range(len(wavelengths)):
    ax6.plot([(coords['source_pulse_t_0'] + (0.5 * coords['source_pulse_length'])).value,
              arrival_times['wavelength', i].value],
             [0.0, sc.norm(coords['position']).value], color='r')
fig6

Taking a closer look at the WFM choppers,
we observe that the numbers we picked for the first two neutrons actually lead to unphysical paths:
they do not make it through the chopper openings!

In [ ]:
del ax6.texts[3:]
ax6.set_xlim(-1.0e3, 2.0e4)
ax6.set_ylim(-1.5, 10.0)
fig6.canvas.draw_idle()
fig6

So we modify our values so that neutrons 1 and 2 make it through:

In [ ]:
wavelengths = sc.array(dims=['wavelength'],
                       values=[1.75, 3.2, 4.5, 6.0, 7.0, 8.25],
                       unit='angstrom')
arrival_times = sc.to_unit(
    alpha * dz * wavelengths,
    'us') + coords['source_pulse_t_0'] + (0.5 * coords['source_pulse_length'])

In [ ]:
fig7 = wfm.plot.time_distance_diagram(da)
ax7 = fig7.get_axes()[0]

for i in range(len(wavelengths)):
    ax7.plot([(coords['source_pulse_t_0'] + (0.5 * coords['source_pulse_length'])).value,
              arrival_times['wavelength', i].value],
             [0.0, sc.norm(coords['position']).value], color='r')
del ax7.texts[3:]
ax7.set_xlim(-1.0e3, 2.0e4)
ax7.set_ylim(-1.5, 10.0)
fig7

And repeat the stitching process:

In [ ]:
tmin = sc.min(arrival_times)
tmax = sc.max(arrival_times)
dt = 0.1 * (tmax - tmin)
coords['time'] = sc.linspace(dim='time',
                             start=(tmin - dt).value,
                             stop=(tmax + dt).value,
                             num=2001,
                             unit=dt.unit)
counts, _ = np.histogram(arrival_times.values, bins=coords['time'].values)
da = sc.DataArray(coords=coords,
                  data=sc.array(dims=['time'], values=counts, unit='counts'))
stitched = wfm.stitch(frames=frames, data=da, dim='time', bins=2001)
wav = stitched.transform_coords("wavelength", graph=graph)

In [ ]:
wav["wavelength", (1.55*sc.units.angstrom):(1.95*sc.units.angstrom)].plot()

This time, the peak is much closer to 1.75 &#8491;,
and we can make sure the sum within the $\Delta\lambda$ range is 1 for all 6 neutrons:

In [ ]:
for i in range(len(wavelengths)):
    lam = wavelengths["wavelength", i]
    half_dlam = 0.5 * dlambda_over_lambda * lam
    print("Lambda:", lam, ", count in range:",
          sc.sum(wav['wavelength', lam - half_dlam:lam + half_dlam]).value)